In [ ]:
import numpy as np

In [ ]:
path_train='/content/drive/My Drive/Colab_data/Data_Mining/Data-Mining-Project/dataset/train.csv'
path_test='/content/drive/My Drive/Colab_data/Data_Mining/Data-Mining-Project/dataset/test.csv'

In [ ]:
import pandas as pd
train = pd.read_csv(path_train)
test = pd.read_csv(path_test)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_x, val_x, train_y, val_y = train_test_split(train['comment_text'],train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']], test_size=0.2, random_state=2)

In [ ]:
import re, string
symbols = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return symbols.sub(r' \1 ', s).split()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
transform_function = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1).fit(train['comment_text'])

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [ ]:
comments_train = transform_function.transform(train_x)
comments_val = transform_function.transform(val_x)
comments_test = transform_function.transform(test['comment_text'])

In [ ]:
train_x = pd.DataFrame(train_x)
val_x = pd.DataFrame(val_x)

In [ ]:
combined = [train_x, val_x, test]

In [ ]:
col = ['total_length', 'capitals', 'caps_vs_length','num_exclamation_marks', 'num_question_marks', 'num_punctuation','num_symbols', 'num_words', 'num_unique_words', 'words_vs_unique','num_smilies']

In [ ]:
for data in combined:
    data['total_length'] = data['comment_text'].apply(len)
    data['capitals'] = data['comment_text'].apply(lambda x: sum(1 for c in x if c.isupper()))
    data['caps_vs_length'] = data.apply(lambda row: float(row['capitals'])/float(row['total_length']),
                                axis=1)
    data['num_exclamation_marks'] = data['comment_text'].apply(lambda x: x.count('!'))
    data['num_question_marks'] = data['comment_text'].apply(lambda x: x.count('?'))
    data['num_punctuation'] = data['comment_text'].apply(lambda x: sum(x.count(w) for w in '.,;:'))
    data['num_symbols'] = data['comment_text'].apply(lambda x: sum(x.count(w) for w in '*&$%'))
    data['num_words'] = data['comment_text'].apply(lambda x: len(x.split()))
    data['num_unique_words'] = data['comment_text'].apply(lambda x: len(set(w for w in x.split())))
    data['words_vs_unique'] = data['num_unique_words'] / data['num_words']
    data['num_smilies'] = data['comment_text'].apply(lambda x: sum(x.count(w) for w in (':-)', ':)', ';-)', ';)')))

In [ ]:
import scipy
train_x = scipy.sparse.csr_matrix(train_x[col].values)
val_x = scipy.sparse.csr_matrix(val_x[col].values)
test = scipy.sparse.csr_matrix(test[col].values)

In [ ]:
comments_train = scipy.sparse.hstack([train_x.tocsr(),comments_train.tocsr()])
comments_val = scipy.sparse.hstack([val_x,comments_val])
comments_test = scipy.sparse.hstack([test,comments_test])

In [ ]:
import xgboost as xgb

In [ ]:
def run(train_X, train_y, test_X, test_y=None, feature_names=None):
    dic = {}
    dic['objective'] = 'binary:logistic'
    dic['eta'] = 0.1
    dic['max_depth'] = 6
    dic['silent'] = 1
    dic['eval_metric'] = 'auc'
    dic['min_child_weight'] = 1
    dic['subsample'] = 0.7
    dic['colsample_bytree'] = 0.7
    num = 100
    list_dic = list(dic.items())

    xgtrain = xgb.DMatrix(train_X, label=train_y)
    xgtest = xgb.DMatrix(test_X, label=test_y)

    model = xgb.train(list_dic, xgtrain, num, [ (xgtrain,'train'), (xgtest, 'test') ], early_stopping_rounds=10)

    return model 

In [ ]:
col = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
preds = np.zeros((test.shape[0], len(col)))

for i, j in enumerate(col):
    print('fit '+j)
    model = run(comments_train, train_y[j], comments_val,val_y[j])
    preds[:,i] = model.predict(xgb.DMatrix(comments_test), ntree_limit = model.best_ntree_limit)
    gc.collect()

fit toxic
[0]	train-auc:0.701894	test-auc:0.699906
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 10 rounds.
[1]	train-auc:0.720606	test-auc:0.71772
[2]	train-auc:0.761526	test-auc:0.756648
[3]	train-auc:0.812048	test-auc:0.803294
[4]	train-auc:0.820223	test-auc:0.813418
[5]	train-auc:0.82538	test-auc:0.81726
[6]	train-auc:0.853074	test-auc:0.845996
[7]	train-auc:0.85293	test-auc:0.845771
[8]	train-auc:0.864441	test-auc:0.855824
[9]	train-auc:0.873903	test-auc:0.866073
[10]	train-auc:0.881353	test-auc:0.873859
[11]	train-auc:0.885842	test-auc:0.879595
[12]	train-auc:0.887051	test-auc:0.881328
[13]	train-auc:0.894386	test-auc:0.889244
[14]	train-auc:0.899532	test-auc:0.894405
[15]	train-auc:0.903192	test-auc:0.897181
[16]	train-auc:0.904132	test-auc:0.896953
[17]	train-auc:0.906056	test-auc:0.898806
[18]	train-auc:0.906139	test-auc:0.89896
[19]	train-auc:0.909253	test-auc:0.901529
[20]	train-auc:0.911407	

In [ ]:
labels=pd.read_csv('/content/drive/My Drive/Colab_data/Data_Mining/Data-Mining-Project/dataset/test_labels.csv')
labels=np.array(labels.iloc[:,1:])
sum_labels=np.sum(labels,axis=1)
idx=sum_labels>=0

In [ ]:
preds_consider=preds[idx]
labels_consider= labels[idx]
preds_consider.shape,labels_consider.shape

((63978, 6), (63978, 6))

In [ ]:
from sklearn.metrics import roc_auc_score
scores=[]
for i in range(6):
  scores.append(roc_auc_score(labels_consider[:,i],preds_consider[:,i]))
np.mean(scores)

0.9635699477762086